In [1]:
import os
import random
import numpy as np
import pandas as pd
from instructblip import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import Dataset, DataLoader
from pytorch_metric_learning import losses
from PIL import Image
import optuna
from optuna.trial import TrialState
from tqdm.notebook import tqdm

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

In [5]:
def file_to_embed(embeds, file):
    emb = []
    for f in file:
        emb.append(embeds[f])
    return torch.stack(emb)

In [10]:
class AdienceDataset(Dataset):
    def __init__(self, annot_file, img_dir):
        self.img_lbls = pd.read_csv(annot_file, header=None)
        self.img_dir = img_dir
    
    def __len__(self):
        return len(self.img_lbls)
    
    def __getitem__(self, idx):
        img_file = self.img_lbls.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_file)
        image = Image.open(img_path)
        label = self.img_lbls.iloc[idx, 1]

        positive_list = self.img_lbls[self.img_lbls.iloc[:, 1] == label].index.values
        positive_list = np.setdiff1d(positive_list, np.array([idx]))
        positive_item = random.choice(positive_list)
        positive_file = self.img_lbls.iloc[positive_item, 0]

        negative_list = self.img_lbls[self.img_lbls.iloc[:, 1] != label].index.values
        negative_item = random.choice(negative_list)
        negative_file = self.img_lbls.iloc[negative_item, 0]

        # return image, label, img_file
        return label, img_file, positive_file, negative_file

In [11]:
train_data = AdienceDataset("../train.csv", "../cropped_Adience/")
val_data = AdienceDataset("../val.csv", "../cropped_Adience/")

In [12]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [13]:
class CombinedLoss(nn.Module):
    def __init__(self, beta=1.0):
        super(CombinedLoss, self).__init__()
        self.beta = beta
        self.triplet = TripletLoss(margin=1.0)
        self.classification = nn.CrossEntropyLoss()
        
    def forward(self, anchor, positive, negative, classification_out, labels):
        triplet_loss = self.triplet(anchor, positive, negative)
        classification_loss = self.classification(classification_out, labels)
        total_loss = (self.beta * triplet_loss) + classification_loss
        
        return total_loss

In [14]:
class InstructBLIPplus(nn.Module):
    def __init__(self):
        super(InstructBLIPplus, self).__init__()
        
        self.fc1 = nn.Linear(in_features=768, out_features=768)
        self.fc2 = nn.Linear(in_features=768, out_features=2)
        
    def forward(self, x):
        x = self.fc1(x)
        x_embed = x
        x_classification = self.fc2(x)
        
        return x_embed, x_classification

In [9]:
embeds = {}
prompt = ""

with torch.no_grad():
    count = 0
    print("train data")
    for img, label, file in train_data:
        inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
        embeds[file] = model.generate(device,
                                      **inputs,
                                      do_sample=False,
                                      num_beams=5,
                                      max_length=256,
                                      min_length=1,
                                      top_p=0.9,
                                      repetition_penalty=1.5,
                                      length_penalty=1.0,
                                      temperature=1)[0]
        count += 1
        if count % 1000 == 0:
            print(count, "images done")

    count = 0
    print("val data")
    for img, label, file in val_data:
        inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
        embeds[file] = model.generate(device,
                                      **inputs,
                                      do_sample=False,
                                      num_beams=5,
                                      max_length=256,
                                      min_length=1,
                                      top_p=0.9,
                                      repetition_penalty=1.5,
                                      length_penalty=1.0,
                                      temperature=1)[0]
        count += 1
        if count % 1000 == 0:
            print(count, "images done")

train data
1000 images done
2000 images done
3000 images done
4000 images done
5000 images done
6000 images done
7000 images done
8000 images done
9000 images done
10000 images done
11000 images done
12000 images done
13000 images done
val data
1000 images done


In [15]:
best_accu = 0.0
def objective(trial):
    model_xtr = InstructBLIPplus().to(device)

    loss_lr = trial.suggest_float("loss_learning_rate", 1e-4, 1e-2, log=True)
    arc_margin = losses.ArcFaceLoss(2, 768).to(device)
    cos_margin = losses.CosFaceLoss(2, 768).to(device)
    loss_optimizer_1 = opt.AdamW(arc_margin.parameters(), lr=loss_lr)
    loss_optimizer_2 = opt.AdamW(cos_margin.parameters(), lr=loss_lr)
    
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    wd = trial.suggest_float('weight_decay', 1e-4, 1e-1, log=True)
    optimizer = opt.AdamW(model_xtr.parameters(), lr=lr, weight_decay=wd)

    beta = trial.suggest_float('beta', 0.1, 1.0, step=0.1)
    gamma = trial.suggest_float('gamma', 0.1, 1.0, step=0.1)
    theta = trial.suggest_float('theta', 0.1, 1.0, step=0.1)
    criterion = CombinedLoss(beta=beta)
    
    batch_size = trial.suggest_int('batch_size', 50, 300)
    num_epochs = 20

    print("Learning rate for Loss: "+ str(loss_lr))
    print("Learning rate: "+ str(lr))
    print("Weight decay: "+ str(wd))
    print("Beta: "+ str(beta))
    print("Gamma: "+ str(gamma))
    print("Theta: "+ str(theta))
    print("Batch size: "+ str(batch_size))
    print("Number of epochs: "+ str(num_epochs))
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
        
        # training loop
        running_loss = []
        running_accu = []
        
        model_xtr.train()
        for label, file, pos_file, neg_file in tqdm(train_loader, desc="Training", leave=False):
            label = label.to(device)

            x1 = file_to_embed(embeds, file)
            x1 = x1.mean(dim=1)
            x2 = file_to_embed(embeds, pos_file)
            x2 = x2.mean(dim=1)
            x3 = file_to_embed(embeds, neg_file)
            x3 = x3.mean(dim=1)
            
            optimizer.zero_grad()
            embed, output = model_xtr(x1)
            pos, _ = model_xtr(x2)
            neg, _ = model_xtr(x3)
            
            pred = torch.argmax(output, 1)
            accuracy = torch.eq(pred, label).sum() / len(label)

            class_triplet_loss = criterion(embed, pos, neg, output, label)
            arc_loss = arc_margin(embed, label)
            cos_loss = cos_margin(embed, label)
            loss = class_triplet_loss + (gamma * arc_loss) + (theta * cos_loss)
            loss.backward()
            loss_optimizer_1.step()
            loss_optimizer_2.step()
            optimizer.step()

            running_accu.append(accuracy.cpu().detach().numpy())
            running_loss.append(loss.cpu().detach().numpy())
        print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
        
        # validation loop
        val_loss = []
        val_accu = []

        model_xtr.eval()
        with torch.no_grad():
            for label, file, pos_file, neg_file in tqdm(val_loader):
                label = label.to(device)
                
                x1 = file_to_embed(embeds, file)
                x1 = x1.mean(dim=1)
                x2 = file_to_embed(embeds, pos_file)
                x2 = x2.mean(dim=1)
                x3 = file_to_embed(embeds, neg_file)
                x3 = x3.mean(dim=1)
                
                embed, output = model_xtr(x1)
                pos, _ = model_xtr(x2)
                neg, _ = model_xtr(x3)
                
                pred = torch.argmax(output, 1)
                accuracy = torch.eq(pred, label).sum() / len(label)
                
                class_triplet_loss = criterion(embed, pos, neg, output, label)
                arc_loss = arc_margin(embed, label)
                cos_loss = cos_margin(embed, label)
                loss = class_triplet_loss + (gamma * arc_loss) + (theta * cos_loss)
                
                val_accu.append(accuracy.cpu().detach().numpy())
                val_loss.append(loss.cpu().detach().numpy())
        val_accu = np.mean(val_accu)
        val_loss = np.mean(val_loss)
        print("Val Loss: {:.4f} - Val Accuracy: {:.4f}".format(val_loss, val_accu))
        
        trial.report(val_accu, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    global best_accu
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model_xtr.state_dict(), "../iblip_qf_triplet_arcface_cosface.pt")
            
    return val_accu

In [16]:
study = optuna.create_study(direction='maximize',
                            study_name='instructblip-triplet-arcface-cosface-qf-study',
                            storage='sqlite:///study5.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=20)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

# Display the study statistics
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-12-30 17:17:41,183] A new study created in RDB with name: instructblip-triplet-arcface-cosface-qf-study


Learning rate for Loss: 0.00023317527963949536
Learning rate: 7.343752487996306e-05
Weight decay: 0.013140653936030252
Beta: 1.0
Gamma: 0.7000000000000001
Theta: 1.0
Batch size: 217
Number of epochs: 20


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/65 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch: 1/20 - Loss: 7.1330 - Accuracy: 0.8314


  0%|          | 0/8 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Val Loss: 4.7955 - Val Accuracy: 0.8772

Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  10
  Number of complete trials:  10


In [17]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.9131198525428772
  Params: 
    loss_learning_rate: 0.0037180569971946987
    learning_rate: 0.0047326093704452964
    weight_decay: 0.00028558538274471015
    beta: 0.5
    gamma: 0.8
    theta: 0.30000000000000004
    batch_size: 295
